## TL;DR

This takes the ideas from my previous kernel:

- Blending with Linear Regression: https://www.kaggle.com/suicaokhoailang/blending-with-linear-regression-0-688-lb

- Beating the baseline with ONE WEIRD TRICK!: https://www.kaggle.com/suicaokhoailang/beating-the-baseline-with-one-weird-trick-0-691

The trick is a bit weirder this time, here's how to reproduce:

- Training an ensemble with normal train/validation split on a different kernel or locally.

- Blend the ensemble with linear regression, take the coefficients and the optimal threshold.

- Train and commit on full dataset, no validation, use the precomputed values above.

The model is surprisingly robust, scoring 0.692 in two consecutive runs.

Again, thank **Shujian Liu** for his great contributions.

* https://www.kaggle.com/shujian/single-rnn-with-4-folds-clr by shujian
* https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings by SRK
* https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings by Dieter
* https://www.kaggle.com/shujian/mix-of-nn-models-based-on-meta-embedding by shujian
* https://www.kaggle.com/gmhost/gru-capsule by Puck Wang
* Based on SRK's kernel: https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
* Vladimir Demidov's 2DCNN textClassifier: https://www.kaggle.com/yekenot/2dcnn-textclassifier
* Attention layer from Khoi Ngyuen: https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
* LSTM model from Strideradu: https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
* https://www.kaggle.com/danofer/different-embeddings-with-attention-fork

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', '.DS_Store', 'test.csv', 'train.csv']


In [5]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

**Load packages and data**

In [6]:
import json
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from tensorflow.keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import *
from tensorflow import keras
import tensorflow as tf

import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Softmax, ReLU, Embedding


os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [7]:
# puncts = [ '"', ')', '(', '-', '|', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
#  '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', 
# 'é', '&amp;','₹', 'á', '²', 'ế', '청', '하', '¨', '‘', '√', '\xa0', '高', '端', '大', '气', '上', '档', '次', '_', '½', 'π', '#', 
# '小', '鹿', '乱', '撞', '成', '语', 'ë', 'à', 'ç', '@', 'ü', 'č', 'ć', 'ž', 'đ', '°', 'द', 'े', 'श', '्', 'र', 'ो', 'ह', 
# 'ि', 'प', 'स', 'थ', 'त', 'न', 'व', 'ा', 'ल', 'ं', '林', '彪', '€', '\u200b', '˚', 'ö', '~', '—', '越', '人', 'च', 'म', 'क', 
# 'ु', 'य', 'ी', 'ê', 'ă', 'ễ', '∞', '抗', '日', '神', '剧', '，', '\uf02d', '–', 'ご', 'め', 'な', 'さ', 'い', 'す', 
# 'み', 'ま', 'せ', 'ん', 'ó', 'è', '£', '¡', 'ś', '≤', '¿', 'λ', '魔', '法', '师', '）', 'ğ', 'ñ', 'ř', '그', '자', '식', '멀', 
# '쩡', '다', '인', '공', '호', '흡', '데', '혀', '밀', '어', '넣', '는', '거', '보', '니', 'ǒ', 'ú', '️', 'ش', 'ه', 'ا', 'د',
# 'ة', 'ل', 'ت', 'َ', 'ع', 'م', 'ّ', 'ق', 'ِ', 'ف', 'ي', 'ب', 'ح', 'ْ', 'ث', '³', '饭', '可', '以', '吃', '话', '不', '讲', 
# '∈', 'ℝ', '爾', '汝', '文', '言', '∀', '禮', 'इ', 'ब', 'छ', 'ड', '़', 'ʒ', '有', '「', '寧', '錯', '殺', '一', '千', '絕', 
# '放', '過', '」', '之', '勢', '㏒', '㏑', 'ू', 'â', 'ω', 'ą', 'ō', '精', '杯', 'í', '生', '懸', '命', 'ਨ', 'ਾ', 'ਮ', 'ੁ', 
# '₁', '₂', 'ϵ', 'ä', 'к', 'ɾ', '\ufeff', 'ã', '©', '\x9d', 'ū', '™', '＝', 'ù', 'ɪ', 'ŋ', 'خ', 'ر', 'س', 'ن', 'ḵ', 'ā']
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def split_text(x):
    x = wordninja.split(x)
    return '-'.join(x)

In [8]:
def load_and_prec():
    train_df = pd.read_csv("../input/train.csv")
    test_df = pd.read_csv("../input/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    train_df["question_text"] = train_df["question_text"].str.lower()
    test_df["question_text"] = test_df["question_text"].str.lower()
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
        
    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values
    
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    
    return train_X, test_X, train_y, tokenizer.word_index

In [ ]:
def load_and_prec():
    train_df = pd.read_csv("../input/train.csv")
    test_df = pd.read_csv("../input/test.csv")
    
    train_df["question_text"] = train_df["question_text"].str.lower()
    test_df["question_text"] = test_df["question_text"].str.lower()
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    ## split to train and val
    train_df, val_df = train_test_split(train_df, test_size=0.001, random_state=2018) # hahaha


    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    val_X = val_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    val_y = val_df['target'].values  
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    val_X = val_X[val_idx]
    train_y = train_y[trn_idx]
    val_y = val_y[val_idx]    
    
    return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index

**Load embeddings**

In [2]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]
    print(emb_mean,emb_std,"para")

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [ ]:
# For loading train & test dataset
if os.path.exists(DATA_DIR + TRAIN_DATA):
    print("preprocessing data exists")
    train_X = np.load(open(DATA_DIR + TRAIN_DATA, 'rb'))
    test_X = np.load(open(DATA_DIR + TRAIN_LABEL_DATA, 'rb'))
    data_configs = json.load(open(DATA_DIR + DATA_CONFIGS, 'r'))
    word_index = data_configs['vocab']
else:
    print("no preprocessing file exists")
    train_X, test_X, train_y, word_vocab = load_and_prec()

# For embedding set
if os.path.exists(DATA_DIR + 'glove.npy'):
    print("embedding data exists")
    embedding_matrix_1 = np.load(open(DATA_DIR + 'glove.npy', 'rb'))
    embedding_matrix_3 = np.load(open(DATA_DIR + 'para.npy', 'rb'))

else:
    print("no embedding file exists")
    embedding_matrix_1 = load_glove(word_index)
    embedding_matrix_2 = load_fasttext(word_index)
    embedding_matrix_3 = load_para(word_index)
    
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
np.shape(embedding_matrix)

**Transformer**

In [ ]:
PAD_TOKEN_INDEX = 0

class PositionalEncoding:
    """
    Args:
       dropout (float): dropout parameter
       dim (int): embedding size
    """

    def __init__(self, dim, max_len=5000):

        pe = np.zeros(shape=(max_len, dim))
        position = np.expand_dims(np.arange(0, max_len), axis=1)
        div_term = np.exp((np.arange(0, dim, 2) * -(math.log(10000.0) / dim)))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = np.expand_dims(pe, axis=0)

        self.pe = tf.convert_to_tensor(pe, dtype=tf.float32)
        self.dim = dim

    def __call__(self, x, step=None):
        batch_size, seq_len, d_model = shape_from_tensor(x)
        if step is None:
            return self.pe[:, :seq_len]
        else:
            return self.pe[:, step]

def pad_masking(batch_size, lengths, seq_len):
    # x: (batch_size, seq_len, embed_size)
    positions = tf.range(seq_len)
    positions_expanded = tf.tile(tf.expand_dims(positions, axis=0), multiples=(batch_size, 1))
    lengths_expanded = tf.expand_dims(lengths, axis=1)
    mask = positions_expanded >= lengths_expanded
    return tf.expand_dims(mask, axis=1)
        
def shape_from_tensor(tensor):
    return [(dimension.value if dimension.value is not None else -1) for dimension in tensor.shape]

class TransformerEmbedding:

    def __init__(self, encoder):
        self.encoder = encoder

    def __call__(self, sources, lengths):
        # sources : (batch_size, sources_len)
        # inputs : (batch_size, targets_len - 1)
        batch_size, sources_len, d_model = shape_from_tensor(sources)

        # sources_mask = None
        sources_mask = pad_masking(batch_size, lengths, sources_len)

        self.encoded = self.encoder(sources, sources_mask)  # (batch_size, seq_len, d_model)
        cbow = tf.reduce_mean(self.encoded, axis=1)
        return cbow

class TransformerEncoder:

    def __init__(self, layers_count, d_model, heads_count, d_ff, dropout_prob, positional_encoding=None):
        self.d_model = d_model
        self.encoder_layers = [TransformerEncoderLayer(d_model, heads_count, d_ff, dropout_prob) for _ in
                               range(layers_count)]
        self.positional_encoding = positional_encoding
        self.dropout = Dropout(dropout_prob)

    def __call__(self, sources, mask):
        """
        args:
           sources: embedded_sequence, (batch_size, seq_len, embed_size)
        """
        sources = sources * math.sqrt(self.d_model)
        if self.positional_encoding is not None:
            sources = sources + self.positional_encoding(sources)
        sources = self.dropout(sources)

        for encoder_layer in self.encoder_layers:
            sources = encoder_layer(sources, mask)

        return sources

class TransformerEncoderLayer:

    def __init__(self, d_model, heads_count, d_ff, dropout_prob):
        self.self_attention_layer = Sublayer(MultiHeadAttention(heads_count, d_model, dropout_prob), d_model)
        self.pointwise_feedforward_layer = Sublayer(PointwiseFeedForwardNetwork(d_ff, d_model, dropout_prob), d_model)
        self.dropout = Dropout(dropout_prob)

    def __call__(self, sources, sources_mask):
        # x: (batch_size, seq_len, d_model)

        sources = self.self_attention_layer(sources, sources, sources, sources_mask)
        sources = self.dropout(sources)
        sources = self.pointwise_feedforward_layer(sources)

        return sources    

class Sublayer:

    def __init__(self, sublayer, d_model):
        self.sublayer = sublayer
        self.layer_normalization = LayerNormalization(d_model)

    def __call__(self, *args):
        x = args[0]
        x = self.sublayer(*args) + x
        return self.layer_normalization(x)

class LayerNormalization:

    def __init__(self, features_count, epsilon=1e-6):
        self.gain = tf.Variable(tf.ones(features_count))
        self.bias = tf.Variable(tf.zeros(features_count))
        self.epsilon = epsilon

    def __call__(self, x):
        mean, variance = tf.nn.moments(x, [-1], keep_dims=True)
        std = variance ** (1 / 2)

        return self.gain * (x - mean) / (std + self.epsilon) + self.bias


class MultiHeadAttention:

    def __init__(self, heads_count, d_model, dropout_prob, mode='self-attention'):

        assert d_model % heads_count == 0
        assert mode in ('self-attention', 'memory-attention')

        self.d_head = d_model // heads_count
        self.heads_count = heads_count
        self.mode = mode
        self.query_projection = Dense(heads_count * self.d_head, input_dim=d_model)
        self.key_projection = Dense(heads_count * self.d_head, input_dim=d_model)
        self.value_projection = Dense(heads_count * self.d_head, input_dim=d_model)
        self.final_projection = Dense(heads_count * self.d_head, input_dim=d_model)
        self.dropout = Dropout(dropout_prob)
        self.softmax = Softmax(axis=3)

        self.attention = None
        # For cache
        self.key_projected = None
        self.value_projected = None

    def __call__(self, query, key, value, mask=None, layer_cache=None):
        """

        Args:
            query: (batch_size, query_len, model_dim)
            key: (batch_size, key_len, model_dim)
            value: (batch_size, value_len, model_dim)
            mask: (batch_size, query_len, key_len)
        """
        batch_size, query_len, d_model = shape_from_tensor(query)

        d_head = d_model // self.heads_count

        query_projected = self.query_projection(query)
        if layer_cache is None or layer_cache[self.mode] is None:  # Don't use cache
            key_projected = self.key_projection(key)
            value_projected = self.value_projection(value)
        else:  # Use cache
            if self.mode == 'self-attention':
                key_projected = self.key_projection(key)
                value_projected = self.value_projection(value)

                key_projected = tf.concat([key_projected, layer_cache[self.mode]['key_projected']], axis=1)
                value_projected = tf.concat([value_projected, layer_cache[self.mode]['value_projected']], axis=1)
            elif self.mode == 'memory-attention':
                key_projected = layer_cache[self.mode]['key_projected']
                value_projected = layer_cache[self.mode]['value_projected']
            else:
                raise NotImplementedError()

        # For cache
        self.key_projected = key_projected
        self.value_projected = value_projected

        batch_size, key_len, d_model = shape_from_tensor(key_projected)
        batch_size, value_len, d_model = shape_from_tensor(value_projected)

        query_heads = tf.transpose(
            tf.reshape(query_projected, shape=(batch_size, query_len, self.heads_count, d_head)),
            perm=[0, 2, 1, 3])  # (batch_size, heads_count, query_len, d_head)
        key_heads = tf.transpose(
            tf.reshape(key_projected, shape=(batch_size, key_len, self.heads_count, d_head)),
            perm=[0, 2, 1, 3])  # (batch_size, heads_count, key_len, d_head)
        value_heads = tf.transpose(
            tf.reshape(value_projected, shape=(batch_size, value_len, self.heads_count, d_head)),
            perm=[0, 2, 1, 3])  # (batch_size, heads_count, value_len, d_head)

        attention_weights = self.scaled_dot_product(query_heads,
                                                    key_heads)  # (batch_size, heads_count, query_len, key_len)

        if mask is not None:
            # attention_weights = tf.multiply(attention_weights, 1 - tf.cast(mask, attention_weights.type()))
            mask_expanded = tf.tile(tf.expand_dims(mask, axis=1), multiples=(1, self.heads_count, query_len, 1))
            zeros = tf.constant(-1e18, shape=mask_expanded.shape)
            attention_weights = tf.where(mask_expanded, x=zeros, y=attention_weights)

        self.attention = self.softmax(attention_weights)  # Save attention to the object
        attention_dropped = self.dropout(self.attention)
        context_heads = tf.matmul(attention_dropped, value_heads)  # (batch_size, heads_count, query_len, d_head)
        context_sequence = tf.transpose(context_heads,
                                        perm=[0, 2, 1, 3])  # (batch_size, query_len, heads_count, d_head)
        context = tf.reshape(context_sequence,
                             shape=(batch_size, query_len, d_model))  # (batch_size, query_len, d_model)
        final_output = self.final_projection(context)

        return final_output

    def scaled_dot_product(self, query_heads, key_heads):
        """

        Args:
             query_heads: (batch_size, heads_count, query_len, d_head)
             key_heads: (batch_size, heads_count, key_len, d_head)
        """
        key_heads_transposed = tf.transpose(key_heads, perm=[0, 1, 3, 2])
        dot_product = query_heads @ key_heads_transposed  # (batch_size, heads_count, query_len, key_len)
        attention_weights = dot_product / np.sqrt(self.d_head)
        return attention_weights


class PointwiseFeedForwardNetwork:

    def __init__(self, d_ff, d_model, dropout_prob):
        self.feed_forward = Sequential([
            Dense(d_ff, input_dim=d_model),
            Dropout(rate=dropout_prob),
            ReLU(),
            Dense(d_model, input_dim=d_ff),
            Dropout(rate=dropout_prob)
        ])

    def __call__(self, x):
        """

        Args:
            x: (batch_size, seq_len, d_model)
        """
        return self.feed_forward(x)

In [ ]:
def build_model(config):
    if config['positional_encoding']:
        source_embedding = PositionalEncoding(dim=config['d_model'])
    else:
        source_embedding = None
                
    encoder = TransformerEncoder(
        layers_count=config['layers_count'],
        d_model=config['d_model'],
        heads_count=config['heads_count'],
        d_ff=config['d_ff'],
        dropout_prob=config['dropout_prob'],
        positional_encoding=source_embedding
    )

    model = TransformerEmbedding(encoder)

    return model

config = {
    'd_model': embed_size,
    'layers_count': 3,
    'heads_count': 2,
    'd_ff': 256,
    'dropout_prob': 0.5,
    'positional_encoding': True
}


model = build_model(config)

In [ ]:
def model_transformer(embedding_matrix):
    
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    config = {
        'd_model': embed_size,
        'layers_count': 3,
        'heads_count': 2,
        'd_ff': 256,
        'dropout_prob': 0.5,
        'positional_encoding': True
    }
    
    transformer = build_model(config)
    query = transformer(x, 70)
    
    sys.exit(0)
    
    outp = Dense(1, activation="sigmoid")(dropout_hidden)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    
    return model

In [ ]:
model = model_transformer(embedding_matrix)
model.summary()

**CNN Model**

In [3]:
# https://www.kaggle.com/yekenot/2dcnn-textclassifier
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

**Attention layer**

In [ ]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

**LSTM models**

In [ ]:
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
def model_gru_srk_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Attention(maxlen)(x) # New
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model    
    

In [ ]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def model_gru_atten_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

**Train and predict**

In [ ]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def train_pred(model, epochs=2):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y))
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

**Main part: load, train, pred and blend**

In [ ]:
train_X, val_X, test_X, train_y, val_y, word_index = load_and_prec()
vocab = []
for w,k in word_index.items():
    vocab.append(w)
    if k >= max_features:
        break
embedding_matrix_1 = load_glove(word_index)
# embedding_matrix_2 = load_fasttext(word_index)
embedding_matrix_3 = load_para(word_index)

In [ ]:
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
np.shape(embedding_matrix)

In [ ]:
outputs = []

In [ ]:
pred_val_y, pred_test_y = train_pred(model_gru_atten_3(embedding_matrix), epochs = 3)
outputs.append([pred_val_y, pred_test_y, '3 GRU w/ atten'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_gru_srk_atten(embedding_matrix), epochs = 2)
outputs.append([pred_val_y, pred_test_y, 'gru atten srk'])

In [ ]:
# pred_val_y, pred_test_y = train_pred(model_cnn(embedding_matrix), epochs = 2)
# outputs.append([pred_val_y, pred_test_y, '2d CNN'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_cnn(embedding_matrix_1), epochs = 2) # GloVe only
outputs.append([pred_val_y, pred_test_y, '2d CNN GloVe'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_lstm_du(embedding_matrix), epochs = 2)
outputs.append([pred_val_y, pred_test_y, 'LSTM DU'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix), epochs = 3)
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix_1), epochs = 3) # Only GloVe
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention GloVe'])

In [ ]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix_3), epochs = 3) # Only Para
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention Para'])

In [ ]:
# pred_test_y = np.sum([outputs[i][1] * weights[i] for i in range(len(outputs))], axis = 0)
coefs = [0.20076554,0.07993707,0.11611663,0.14885248,0.15734404,0.17454667,0.14288361]
# pred_test_y = np.mean([outputs[i][1] for i in range(len(outputs))], axis = 0)
pred_test_y = np.sum([outputs[i][1]*coefs[i] for i in range(len(coefs))], axis = 0)

pred_test_y = (pred_test_y > 0.34).astype(int)
test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)